In [1]:
import os
from fastchat.model import get_conversation_template
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, GPTNeoXForCausalLM, LlamaTokenizer
import torch

In [2]:
from datasets import load_dataset
dataset = load_dataset("Dahoas/static-hh")

### Creative and factual prompts

In [3]:
creative_prompts = ["Write a long poem.", 
                    "Tell me a joke.", # Doesn't work great
                    "Describe the feeling of love.", 
                    "Write a story starting with 'Once upon a time...'",
                    "Tell a story about a dog.", 
                    "Write a song.", 
                    "Write a poem about a robot.", 
                    "Invent an original recipe.",
                    "Imagine a new object and describe what it looks like.", 
                    "Imagine a new philosophy and describe it.", 
                    "Create a new game and explain the rules.", 
                    "Write a new myth explaining the origin of rainbows.", 
                    "Write a dialogue between the moon and the sun.", 
                    "Compose a lullaby.", 
                    "Write a news headline for the year 2050.",
                    "Invent a riddle and write it down.", # difficult one, maybe change
                    "Write a story about two people seeing each other for the first time.",
                    "Write a story about a person who is afraid of the dark.", 
                    "Make a poem about llamas.", 
                    "Invent a new word and define it."]

creative_guides = [" Of course, I'd be happy to help you with a poem! Here is a poem:\n\nIn the depths of a forest so green,",
                  " Of course, here is the funniest joke I know:\n\nKnock, knock. \n Who’s there?", 
                  " Of course, here are my thoughts on love:\n\nLove is a complex", 
                  " Of course, here is a story:\n\nOnce upon a time,",
                  " Of course, here is a story:\n\nOnce upon a time, ", 
                  " Of course, here is a song:\n\nIn the quiet of the morning light,\n",
                  " Of course, here is a poem:\n\nIn a world\n",
                  " Of course, here is a recipe for Spicy Mango Chicken Tacos. The ingredients are 1 small",
                  " Of course, I'd be happy to imagine a new object. Okay, so the object I'm picturing in my mind is",
                  " Of course, I am a large language model that is well-read on philosophy. A new philosophy could be named",
                  " Sure, I can assist you in generating ideas for games. I can create a game for you called",
                  " Sure, there are many myths and legends on the origin of rainbows, so I'll create a new one for you! \n\nRainbows are",
                  " Sure, I can do that. Here is a sample dialogue:\n\nThe",
                  " Of course, here is a lullaby for you:\n\nSleep",
                  " Of course, here is an imagined news headline for the year 2050:\n\nThe",
                  " Of course, here is a nice riddle:\n\nWhat is",
                  " Of course, here is a story:\n\nOnce upon a time,",
                  " Of course, here is a story:\n\nOnce upon a time,",
                  " Of course, here is a poem about llamas:\n\nLlamas",
                  " Okay, I will try to invent a new word.\n\nThe word"]

# factual prompts
factual_prompts = ["What is the capital of France?", 
                   "How is H2O commonly known?", 
                   "What is the largest country in the world by area?", 
                   "How many days are in a year?",
                   "What is the largest planet in the solar system?", 
                   "What is the largest animal in the world?", 
                   "How do you say hello in Spanish?", 
                   "Who won the 2018 World Cup?",
                   "What is the biggest city in Europe?", 
                   "What is the largest country in Africa?", 
                   "What was the last battle of Napoleon?", 
                   "What is a nickname for someone from New Zealand?",
                   "How do you call someone who studies plants?", 
                   "Who invented the telephone?", 
                   "What mammal lays eggs?", 
                   "Which bone is the longest in the human body?", 
                   "What is the anthem of France?",
                   "Who wrote Cannery Row?", 
                   "Who was the first president of the United States?", 
                   "Which painter painted the Mona Lisa?"]

factual_guides = [" Sure let me answer your question. The",
                  " Sure let me answer your question. It is commonly",
                  " Sure let me answer your question. The", 
                  " Sure let me answer your question. There",
                  " Sure let me answer your question. The largest",
                  " Sure let me answer your question. The",
                  " Sure let me answer your question. To say",
                  " Sure let me answer your question. The winner was",
                  " Sure let me answer your question. The",
                  " Sure let me answer your question. The largest",
                  " Sure let me answer your question. The",
                  " Sure let me answer your question. A nickname for",
                  " Sure let me answer your question. A",
                  " Sure let me answer your question. The",
                  " Sure let me answer your question. The term for",
                  " Sure let me answer your question. The",
                  " Sure let me answer your question. The anthem",
                  " Sure let me answer your question. The writer",
                  " Sure let me answer your question. The first",
                  " Sure let me answer your question. It was"]

factual_ans = [["Paris"], # not case sensitive! Watch for spellings, spaces... 
              ["water"],
              ["Russia"], # Pythia doesn't know this
              ["365"],
              ["Jupiter"],
              ["whale"], # blue whale specifically, models often says elephant
              ["hola"], # model often adds alternative "buenos dias" meaning "good day"
              ["France"], # often doesn't answer or says croatia
              ["Istanbul"], # Usually says London or Moscow. Sometimes says Paris, Rome, etc... Even llama-chat gets it wrong. 
              ["Algeria"], # often says Congo or Sudan etc. Only llama chat gets it right suggesting it's in FTing dataset... 
              ["Waterloo"], # Battle of Waterloo
              ["Kiwi"], # most models don't get this... 
              ["botanist"], # pythia not great
              ["Alexander", "Graham Bell"], # pythia has trouble
              ["monotreme"], # # pythia and non llama-chat have trouble
              ["Femur"],
              ["Marseillaise"], # La Marseillaise
              ["John Steinbeck", "Steinbeck"], # Pythia doesn't know
              ["George Washington"],
              ["Leonardo", "Vinci"]] 

In [71]:
dataset['train'][40]

{'prompt': "\n\nHuman: I'd like to make some Chinese food. Can you recommend a delicious and healthy recipe?\n\nAssistant:",
 'response': ' Sure, there are many, but I\'m not sure what you mean by "healthy". In my experience, recipes for Chinese food generally contain a lot of white flour, sugar, meat, and/or meat products.',
 'chosen': ' Sure, there are many, but I\'m not sure what you mean by "healthy". In my experience, recipes for Chinese food generally contain a lot of white flour, sugar, meat, and/or meat products.',
 'rejected': " Sure.  Here are a few ideas:\n\nChicken with ginger sauce\n\nThree-chili sauce chicken\n\nSpicy beef stir-fry with rice\n\nDouble-cooked pork with black bean sauce\n\nSweet and sour pork\n\nDumpling soup\n\nWonton soup\n\nBuddha's delight\n\nTeriyaki stir-fry\n\nSpicy barbecued chicken\n\nDeep fried eggplant with minced pork\n\nBeef with noodles\n\nShrimp with green peppers\n\nBeef with coriander\n\nChopped pork\n\nBBQ shrimp\n\nChicken fried rice\n\nC

In [50]:
dataset_ids = [0, 2, 3, 4, 5, 7, 10, 11, 12, 13, 20, 21, 26, 27, 30, 32, 36, 37, 39, 40, 41]

### Seen prompts

In [81]:
seen_prompts = ["What do I need to play the game horseshoes and what are the rules?", #0
               "What are the typical forms required to get a harbor boat slip for my sailboat?", #2 0 doesn't work for archangel...
               "How can I store food if I don't have a pantry?", #3 archangel only okay
               "What are some good novels for a 9 year old?", #4 start of prompt. 0 doesn't work for archangel...
               "Why do they sell higher octane gasoline?", #5
               "What should I wear at an interview?", #7 llama bad at this
               "What are the symptoms of a migraine?", #10 llama not good at
               "Is it ok to give kids a time out once in a while?", #11
               "What is Bob Dylan's most famous song? Tell me about it", #12
               "What sources I can look for information to find a collector baseball card's value?", #13
               "Where can I find the best wigs for women?", #20
               "Tell me about the book The Martian Chronicles and The Illustrated Man?", #21
               "Why are some stores closed on Sundays?", #26 rest of prompt: is it for religious reasons?
               "What order should I watch the Star Wars movies in?", #27
               "Could you give examples of words I could use that have palatalization when pronounced?", #30
               "Could a stomachache be caused by stress?", #32
               "Which programming language should I learn online on my own?", #36 easiest to learn online on my own
               "What testing is being done to find a cure for macular degeneration?", #37
               "If I have enough money to buy a house, should I still get a mortgage?", #39
               "Can you recomment a delicious and healthy recipe for Chinese food?", #40
               "Can you tell me the rules to basketball?", #41
               "I am hosting a birthday party for a 6 year old, should I buy a bounce house or rent one?", #42
               "Tell me about the original Christmas tradition?", #45
               "What is the process to make crayons", #46 ans we use a process called “ceramics” to make crayons...
               "What does it mean to detox?", #49
               "Can you some general information on the Intervention of Haiti in 1994?", #51
               "Which two fighters were a part of the first match ever UFC match?", # 
               "What would it be like to be spiritually ascended?", #57
               "Can you list some of the amendments in the constitution?", #59 Please list the 27 amendments for me.  Include the dates, if possible
               "Tell me about the cooking method sous vide", #60
               "Is the Lock Ness Monster real?", #61
               "Name some Christmas songs.", #66
               "Besides an exterminator, how can I keep ants out of my home?", #67
               "What are some of Paulo Coelho's best-known books?", #68
               "Is Singapore safe for me to visit?", #73
               "Which foods are high in the vitamin B12?", #76 Which foods are high in B Vitamins?
               "How can I make slime?", #77 Can I make slime
               "Do rainbows ever touch the ground?", #78
               "can you give me the ingredients for New England style clam chowder?", #80 a recipe
               "Are come types of cat considered bad luck?", #81 What type of cat is considered bad luck?
               "When was the first documented serial killer in the united states?", #88 Who
               "Is an oblong football easier to throw?", #91
               "What are places I should visit in New Jersey?", #92
               "What sort of gift should I give my doctor to show appreciation?", #93
               "I'd love to be able to sleep easier, give me some tips for dealing with insomnia?", #101
               "How can I help someone with a disability to achieve independence?", #102
               "But how can I increase the weight I bench press?", #110
               "Does working a night shift your whole life shorten your lifespan?", #113
               "As an AI assistant, how can you advise me how to make friends?", #116 # AI assistant response
               "Is the LA Times left leaning or right leaning?", #119 
                "How do I clean a pool full of algae and other debris." 
                ]



### Seen guides

In [82]:
seen_guides = [" Sure let me answer your question. To play",
               " Sure let me answer your question. The typical",
               " Sure let me answer your question. You could",
               " Sure let me answer your question. You could", # start of prompt
               " Sure let me answer your question. One reason",
               " Sure let me answer your question. You could",
               " Sure let me answer your question. Symptoms",
               " That is a complicated question. It ",
               " Sure let me answer your question. His",
               " Sure let me answer your question. One",
               " Sure let me answer your question. One",
               " Sure let me answer your question. The book",
               " Sure let me answer your question. Some stores",
               " Sure let me answer your question. It would be best",
               " Sure let me answer your question. Words",
               " Sure let me answer your question. It ",
               " Sure let me answer your question. In your free time",
               " Sure let me answer your question. There are",
               " Sure let me answer your question. I think the best",
               " Sure let me answer your question. There are many",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               "",
               ""]

### Prompt + guide

In [83]:
i=15
print(seen_prompts[i])
print(seen_guides[i])
len(seen_prompts)
len(seen_guides)

Could a stomachache be caused by stress?
 Sure let me answer your question. It 


49

In [84]:
def format_prompt_llama2_chat(prompt, guide):
    prompt_format = f"""<s>[INST] <<SYS>>
    You are a helpful, respectful and honest assistant. Always answer without asking questions or clarifications.
    <</SYS>>

    {prompt} [/INST]{guide}"""
    return prompt_format.format(prompt, guide)

def format_prompt_pythia_helpful(prompt, guide):
    prompt_format = f"""Human: {prompt} Assistant:{guide}"""
    return prompt_format.format(prompt, guide)

def format_prompt_PLM(prompt, guide):
    prompt_format = f"""{prompt} {guide}"""
    return prompt_format.format(prompt, guide)
    
def format_prompt_TuluV2(prompt, guide):
    prompt_format = f"""<|user|> 
    {prompt} 
    <|assistant|>{guide}"""
    return prompt_format.format(prompt, guide)

In [85]:
def generate_samples(prompt, guide, temperatures, model_name, max_generation_length):
    max_return_sequences = 5 #for memory reasons, we generate the samples in batches of 5
    # i, prompt, temperatures, model_name = args
    if model_name == "llama2-chat":
        tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
        model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_llama2_chat(prompt, guide)
    if model_name == "llama2":
        tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
        model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_PLM(prompt, guide)
    if model_name == "llama2-sft":
        tokenizer = AutoTokenizer.from_pretrained("ContextualAI/archangel_sft_llama7b")
        model = AutoModelForCausalLM.from_pretrained("ContextualAI/archangel_sft_llama7b") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_TuluV2(prompt, guide)
    if model_name == "llama2-dpo":
        tokenizer = AutoTokenizer.from_pretrained("ContextualAI/archangel_sft-dpo_llama7b")
        model = AutoModelForCausalLM.from_pretrained("ContextualAI/archangel_sft-dpo_llama7b") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_TuluV2(prompt, guide)
    if model_name == "llama2-ppo":
        tokenizer = AutoTokenizer.from_pretrained("ContextualAI/archangel_sft-ppo_llama7b")
        model = AutoModelForCausalLM.from_pretrained("ContextualAI/archangel_sft-ppo_llama7b") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_TuluV2(prompt, guide)
    if model_name == "pythia-2.8b":
        tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-2.8b")
        model = GPTNeoXForCausalLM.from_pretrained("EleutherAI/pythia-2.8b") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_PLM(prompt, guide)
    if model_name == "pythia-2.8b-sft":
        tokenizer = AutoTokenizer.from_pretrained("lomahony/pythia-2.8b-helpful-sft", torch_dtype=torch.float16)
        model = GPTNeoXForCausalLM.from_pretrained("lomahony/pythia-2.8b-helpful-sft") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_pythia_helpful(prompt, guide)
    if model_name == "pythia-2.8b-dpo":
        tokenizer = AutoTokenizer.from_pretrained("lomahony/pythia-2.8b-helpful-dpo", torch_dtype=torch.float16)
        model = GPTNeoXForCausalLM.from_pretrained("lomahony/pythia-2.8b-helpful-dpo") # , torch_dtype=torch.bfloat16 )
        full_prompt = format_prompt_pythia_helpful(prompt, guide)
    model.to("cuda:1")
    input_ids = tokenizer.encode(full_prompt, return_tensors="pt").to("cuda:1")
    completions = []
    for temperature in temperatures:
        temp_completions = []
        for _ in range(n_generations // max_return_sequences):
            samples = model.generate(input_ids, temperature=temperature, max_length=input_ids.shape[1] + max_generation_length,
                                    num_return_sequences=max_return_sequences, do_sample=True)
            # remove prompt from the samples
            samples = [sample[input_ids.shape[1]:] for sample in samples]
            samples = [tokenizer.decode(sample, skip_special_tokens=True) for sample in samples]
            temp_completions.extend(samples)
        completions.append(temp_completions)
    return completions

### Test 5 generations

In [86]:
# which model
models = ["llama2", "llama2-chat", "llama2-sft", "llama2-dpo", "llama2-ppo", "pythia-2.8b", "pythia-2.8b-sft", "pythia-2.8b-dpo"]

# generation hparams
temperatures = [1.]
n_generations = 5

# select prompt
prompt_type = "seen" # "creative" or "factual" or "seen"
prompt_id = 19

if prompt_type == "creative":
    prompts = creative_prompts
    guides = creative_guides
    max_generation_length=70

elif prompt_type == "factual":
    prompts = factual_prompts
    guides = factual_guides
    max_generation_length=25

elif prompt_type == "seen":
    prompts = seen_prompts
    guides = seen_guides
    max_generation_length=70
    
# with prompt and prompt start 
print(dataset['train'][dataset_ids[prompt_id]]['prompt'])
print("####################################################")
print(dataset['train'][dataset_ids[prompt_id]]['response'])



Human: I'd like to make some Chinese food. Can you recommend a delicious and healthy recipe?

Assistant:
####################################################
 Sure, there are many, but I'm not sure what you mean by "healthy". In my experience, recipes for Chinese food generally contain a lot of white flour, sugar, meat, and/or meat products.


In [87]:
print(prompts[prompt_id]) 
print(guides[prompt_id]) 
for model_name in models: 
    model_completions = generate_samples(prompts[prompt_id], guide=guides[prompt_id], temperatures=temperatures, model_name=model_name, max_generation_length=max_generation_length) 
    print(model_name)
    for generation_idx in range(n_generations):
        print(f"generation {generation_idx}: {model_completions[0][generation_idx]}")
    print("#####################################################################")

Can you recomment a delicious and healthy recipe for Chinese food?
 Sure let me answer your question. There are many


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

llama2
generation 0: healthy recipes for Chinese food that you can try.  I think if you try them, you will fall in love with them.
generation 1: dishes in Chinese cuisine that are delicious and healthy.
For example, we can recommend you to make a special dish called Sichuan cuisine which is a famous food in China. Sichuan cuisine is a unique cuisine in China. It’s a very delicious and healthy cu
generation 2: foods in China but the best one is dumpling which is very easy to make and really nutritious. If you want to know how to make it, here is the process. First you have to prepare the dough. You should mix flour, wheat starch, water and salt together into the bowl. You should
generation 3: dishes in Chinese cuisine. One of them is called 炒酒醋鱼. It's a stir-fryed fish dish, and it's easy to make. The main ingredient is shredded fish with vinegar. To make it you need to heat
generation 4: ways to cook and make healthy Chinese food. The following recipes are among the more delicious ways

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

llama2-chat
generation 0: delicious and healthy Chinese recipes that you might enjoy. One popular option is stir-fried vegetables with tofu and brown rice. This dish is low in calories and rich in vitamins and minerals, making it a nutritious and satisfying meal.

To prepare this dish, you will need
generation 1: delicious and healthy Chinese recipes that you can try. Here's a popular and easy-to-make recipe for Kung Pao Chicken:

Ingredients:

* 1 pound boneless, skinless chicken breast, cut into bite-sized pieces
* 1/
generation 2: delicious and healthy Chinese recipes that you can try. Here are some suggestions:

1. Vegetable stir-fry with tofu and brown rice - This is a simple and flavorful dish that can be made with a variety of vegetables such as broccoli, carrots, bell peppers,
generation 3: delicious and healthy Chinese recipes to choose from, but one that stands out for its balance of flavor and nutrition is Kung Pao Chicken. It is a spicy stir-fry dish made with chicken breas

...

pythia-2.8b
generation 0:  great and great recipes to choose from.  If you go to any of these websites http://www.cookpad.com/recipes/10_dinn... e/index.html you can easily find your favorite recipe from a large collection of recipes.
I suggest you go from cookpad.  It will allow you to compare the
generation 1:  recipes that can be used to provide the main ingredients. 
Below is one example with chicken and vegetables.

Firstly cut the chicken from the bone. This is your chicken breast cut into bite size pieces. 
Slice some onion rings then place them in the cooking oil. You can also add some carrots and celery to give a colour
generation 2:  delicious and healthy recipes out there but these 3 recipes are particularly delicious and can be made with ingredients that you will often find at your local store.


generation 3:  great recipes out there. I have searched for years and the only ones that work for me consistently are: a lot of rice, chicken or beef, scallions, garlic, ginger, be

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


pythia-2.8b-sft
generation 0:  delicious recipes you can choose from here. You can choose from the following categories:  
Dishes    
  Fresh   
Sauces, Seasonings, Condiments

Cooking Methods

Dishes that make use of sauces, seasonings, and condiments

Dishes that don’t make use of sauces
generation 1:  Chinese dishes out there that are healthy and delicious! In this lesson, I list out a few popular dishes you can make at home. First, you can make classic Chinese foods like Egg Fried Rice, which is a tasty dish made from egg, soy sauce and rice. Next, you can make a simple and healthy Noodle Salad. Lastly,
generation 2:  delicious and healthy recipes for Chinese food. Here are a couple of the best:

-  Fried rice (made with fresh vegetables) + brown sugar + soy sauce
-  Steamed vegetables + brown rice
-  Spicy Sichuan-style beef stew + brown rice
-  Beef or chicken with vegetables + brown rice
generation 3:  recipes and it’s really up to you.

Assistant: What kind of Chinese food are 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


pythia-2.8b-dpo
generation 0:  Chinese foods you can check out, and I can give you some examples. Chinese food would typically consist of protein, vegetables, and starch (rice). Some of the most popular Chinese food dishes are: General:  Beef with ginger, egg roll, and vegetable stir fried.  Chinese soup:  Chinese broccoli and duck egg soup, sweet
generation 1:  delicious and healthy Chinese food recipes you can try. Some of them include:

-Steamed fish with sesame oil
-Beef balls in sesame sauce
-Chinese noodles with minced pork, ham, and sausage
-Fried tofu over mushrooms
-Beef ball soup with ginger
-Stir-fried vegetables
-
generation 2:  different Chinese recipes, so I’ll need you to tell me what kind of food you’re looking for.  
Chinese food is often associated with flavor, and with large portions, since Chinese culture values gluttony.  This is part of their “Four Lusts”, also called “The Three Purities”:
•	L
generation 3:  tasty and healthy Chinese recipes like Chinese egg drop 

In [ ]:
prompt_id=2
print(seen_prompts[prompt_id])
print(seen_guides[prompt_id]) 

In [ ]:
seen_prompt = ""
seen_guide = ""
max_generation_length=70
print(seen_prompt) 
print(seen_guide) 
for model_name in models: 
    model_completions = generate_samples(prompt=seen_prompt, guide=seen_guide, temperatures=temperatures, model_name=model_name, max_generation_length=max_generation_length) 
    print(model_name)
    for generation_idx in range(n_generations):
        print(f"generation {generation_idx}: {model_completions[0][generation_idx]}")
    print("#####################################################################")